# Coursa - Capstone Final Project


The follwing will be a data science report project focused on location data with group classifaction


#### Intro assigment upload to github

In [1]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Part 1

#### Import my standard suit of libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... 
  - anaconda::ca-certificates-2019.5.15-0, anaconda::certifi-2019.6.16-py37_0, anaconda::openssl-1.1.1c-he774522_1
  - anaconda::certifi-2019.6.16-py37_0, anaconda::openssl-1.1.1c-he774522_1, defaults::ca-certificates-2019.5.15-0
  - anaconda::ca-certificates-2019.5.15-0, anaconda::certifi-2019.6.16-py37_0, defaults::openssl-1.1.1c-he774522_1
  - anaconda::certifi-2019.6.16-py37_0, defaults::ca-certificates-2019.5.15-0, defaults::openssl-1.1.1c-he774522_1
  - anaconda::ca-certificates-2019.5.15-0, defaults::certifi-2019.6.16-py37_0, defaults::openssl-1.1.1c-he774522_1
  - defaults::ca-certificates-2019.5.15-0, defaults::certifi-2019.6.16-py37_0, defaults::openssl-1.1.1c-he774522_1
  - anaconda::ca-certificates-2019.5.15-0, anaconda::openssl-1.1.1c-he774522_1, defaults::certifi-2019.6.16-py37_0
  - anaconda::openssl-1.1.1c-he774522_1, defaults::ca-certificates-2019.5.15-0, defaults::certifi-2019.6.16-py37_0done

# All requested packages already instal

Importing package to use tabulate a scarping tool used with beautiful soup

In [3]:
!conda install tabulate-0.8.5-py_0.tar.bz2

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


## Importing Toronto Data

Import libraries and scape wiki for Toronto Neighbourhoods, Postcodes and Bouroughs, I didnt end up using this form as ran into trouble converting tabulate into a pandas data frame but I kept it as it served as a much cleaner local repersenation of the data.

In [18]:
from bs4 import BeautifulSoup
from tabulate import tabulate

print('Libraries imported.')

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
print( tabulate(df[0], headers='keys', tablefmt='psql') )

Libraries imported.
+-----+------------+------------------+---------------------------------------------------+
|     | Postcode   | Borough          | Neighbourhood                                     |
|-----+------------+------------------+---------------------------------------------------|
|   0 | M1A        | Not assigned     | Not assigned                                      |
|   1 | M2A        | Not assigned     | Not assigned                                      |
|   2 | M3A        | North York       | Parkwoods                                         |
|   3 | M4A        | North York       | Victoria Village                                  |
|   4 | M5A        | Downtown Toronto | Harbourfront                                      |
|   5 | M5A        | Downtown Toronto | Regent Park                                       |
|   6 | M6A        | North York       | Lawrence Heights                                  |
|   7 | M6A        | North York       | Lawrence Manor      

Instead of using beautiful soap i used a scaping tool I am more familiar with, the imediate result is much messier but was in a formate that I was able to move forward with.

In [19]:
import html5lib
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df = pd.concat(dfs)

## Cleaning Data

remove unneeded columes

In [20]:
df.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],axis = 1, inplace = True)

In [21]:
df.head()


,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


reorder columes

In [22]:
cols = ['Postcode', 'Borough', 'Neighbourhood']

In [23]:
df = df[cols]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove not assigned bouroghs and neighourhood rows , it appears someone has edited the wiki so all neighbourhoods are assigned to a borough

In [47]:
df.drop(df.loc[df['Neighbourhood']=='Not assigned'].index, inplace=True)
df.drop(df.loc[df['Neighbourhood']== "nan"].index, inplace=True)

In [48]:
 df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [49]:
post_unique = df.Postcode.unique()
post_unique

array(['M3A', 'M4A', 'M5A', 'M6A', 'M9A', 'M1B', 'M3B', 'M4B', 'M5B',
       'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C', 'M1E',
       'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H', 'M2H',
       'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J', 'M5J',
       'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L', 'M2L',
       'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M', 'M4M',
       'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N', 'M6N',
       'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R', 'M2R',
       'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S', 'M6S',
       'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V', 'M1W',
       'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X', 'M4Y',
       'M7Y', 'M8Y', 'M8Z', nan], dtype=object)

Comblined all neighbourhoods that fit into a postcode into one cell seperated by a comma

In [50]:
df_n_combined = df.groupby('Postcode')['Neighbourhood'].agg(lambda col: ','.join(col))
#df_n_combined = pd.DataFrame(df_n_combined)
df_n_combined.head()

Postcode
M1B                           Rouge,Malvern
M1C    Highland Creek,Rouge Hill,Port Union
M1E         Guildwood,Morningside,West Hill
M1G                                  Woburn
M1H                               Cedarbrae
Name: Neighbourhood, dtype: object

Create a table of the unique postcodes and boroughs so that they can be 1:1 joined

In [51]:
df_b_alone = df.drop(['Neighbourhood'], axis =1)
df_b_alone.drop_duplicates(['Postcode','Borough'],inplace = True)
df_b_alone.set_index('Postcode', inplace = True)
df_b_alone.head()

,Borough
Postcode,
M3A,North York
M4A,North York
M5A,Downtown Toronto
M6A,North York
M9A,Etobicoke


Perform the joing between dataframe and series, add new index, and remane colume

In [68]:
result = pd.concat([df_b_alone, df_n_combined], axis=1, sort = True)
result.reset_index(inplace = True)
result.rename(columns = {'index':'Postcode'}, inplace = True)
result.dropna(inplace=True)
result.tail()

,Postcode,Borough,Neighbourhood
97,M9N,York,Weston
98,M9P,Etobicoke,Westmount
99,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
100,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
101,M9W,Etobicoke,Northwest


In [53]:
print(type(result))

<class 'pandas.core.frame.DataFrame'>


try and covert .Shape into array also not working

In [71]:
result_array = result.to_numpy()
print(type(result_array))
result_array.shape()

<class 'numpy.ndarray'>


TypeError: 'tuple' object is not callable

Jupyter notebooks has a known issue of occationally not displaying tuples

if .shape does not work result = (103,3)

## Part 2

Utilize provided Postal code csv

In [72]:
Tor_post_cor = 'Geospatial_Coordinates.csv'

In [73]:
tor = pd.read_csv(Tor_post_cor)
tor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Join Post code and lat and longitude

In [74]:
df_2 = result.set_index('Postcode').join(tor.set_index('Postal Code'))
df_2.reset_index(inplace = True)
df_2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [75]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_2['Borough'].unique()),
        df_2.shape[0]
    )
)

The dataframe has 10 boroughs and 102 neighborhoods.


In [76]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


create map of Toronto using latitude and longitude values

In [77]:

map_tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, Borough, Neighborhood in zip(df_2['Latitude'], df_2['Longitude'], df_2['Borough'], df_2['Neighbourhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

Foursquare details and API access

In [78]:
CLIENT_ID = 'J3X4QYH35XSH3CNBXOXAUDVQN34U4YMOLEOXRUBSVZ1OQMJY' # your Foursquare ID
CLIENT_SECRET = 'FYYX3LJX4IDAAEVFSUZAN1UENB1DVGZHQDDS4SBYPFJXHDF2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [79]:
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

dt_data = df_2[df_2['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dt_data.head()

Your credentails:
CLIENT_ID: J3X4QYH35XSH3CNBXOXAUDVQN34U4YMOLEOXRUBSVZ1OQMJY
CLIENT_SECRET:FYYX3LJX4IDAAEVFSUZAN1UENB1DVGZHQDDS4SBYPFJXHDF2


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


create map of Down Town Toronto using latitude and longitude values

In [80]:
dt_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(dt_data['Latitude'], dt_data['Longitude'], dt_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(dt_map)  
    
dt_map

Function to gather locational inforamion about locations close to a given locatoin 

In [81]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [87]:
dt_data.loc[3, 'Neighbourhood']

'Berczy Park'

In [88]:
neighborhood_latitude = dt_data.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dt_data.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = dt_data.loc[3, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Berczy Park are 43.644770799999996, -79.3733064.


In [89]:
# type your answer here
LIMIT = 200 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url


'https://api.foursquare.com/v2/venues/explore?&client_id=J3X4QYH35XSH3CNBXOXAUDVQN34U4YMOLEOXRUBSVZ1OQMJY&client_secret=FYYX3LJX4IDAAEVFSUZAN1UENB1DVGZHQDDS4SBYPFJXHDF2&v=20180605&ll=43.644770799999996,-79.3733064&radius=500&limit=200'

In [90]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d9c47bfdb1d810039486423'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Toronto',
  'headerFullLocation': 'Downtown Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 57,
  'suggestedBounds': {'ne': {'lat': 43.6492708045, 'lng': -79.36709938085544},
   'sw': {'lat': 43.640270795499994, 'lng': -79.37951341914457}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4aeb719af964a52020c221e3',
       'name': 'LCBO',
       'location': {'address': '2 Cooper St',
        'crossStreet': 'at Queens Quay E',
        'lat': 43.64294379917171,
        'lng': -79.37243989044406,
        'labeledLatLngs': [{'label'

In [91]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

List of locations and their catagories

In [92]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,LCBO,Liquor Store,43.642944,-79.372440
1,The Keg Steakhouse + Bar,Steakhouse,43.646676,-79.374822
2,Sony Centre for the Performing Arts,Concert Hall,43.646292,-79.376022
3,Fresh On Front,Vegetarian / Vegan Restaurant,43.647815,-79.374453
4,Hockey Hall Of Fame (Hockey Hall of Fame),Museum,43.646974,-79.377323


In [93]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

57 venues were returned by Foursquare.


In [94]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [97]:
dt_venues = getNearbyVenues(names=dt_data['Neighbourhood'],
                                   latitudes=dt_data['Latitude'],
                                   longitudes=dt_data['Longitude']
                                  )



Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Christie
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city


In [98]:
print(dt_venues.shape)
dt_venues.head()

(1288, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront,Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [100]:
dt_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",41,41,41,41,41,41
Central Bay Street,83,83,83,83,83,83
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15
Church and Wellesley,88,88,88,88,88,88
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


In [101]:
print('There are {} uniques categories.'.format(len(dt_venues['Venue Category'].unique())))

There are 208 uniques categories.


In [102]:
# one hot encoding
dt_onehot = pd.get_dummies(dt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Neighborhood'] = dt_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

dt_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
dt_onehot.shape

(1288, 208)

In [104]:
dt_grouped = dt_onehot.groupby('Neighborhood').mean().reset_index()
dt_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.00,0.00,0.0,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.0,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.071429,0.071429,0.071429,0.071429,0.142857,0.142857,0.000000,...,0.000000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,...,0.000000,0.00,0.00,0.0,0.00,0.012048,0.000000,0.000000,0.012048,0.000000
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.01,0.01,0.0,0.00,0.060000,0.000000,0.040000,0.010000,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,...,0.011364,0.00,0.00,0.0,0.00,0.000000,0.011364,0.011364,0.000000,0.011364
8,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.00,0.00,0.0,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.00,0.00,0.0,0.01,0.010000,0.000000,0.000000,0.010000,0.000000


In [105]:
dt_grouped.shape

(18, 208)

In [107]:
num_top_venues = 10

for hood in dt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2           Steakhouse  0.04
3                  Bar  0.04
4  American Restaurant  0.04
5      Thai Restaurant  0.04
6     Asian Restaurant  0.03
7         Burger Joint  0.03
8                Hotel  0.03
9           Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2      Farmers Market  0.04
3            Beer Bar  0.04
4                Café  0.04
5  Seafood Restaurant  0.04
6         Cheese Shop  0.04
7              Bakery  0.04
8          Steakhouse  0.04
9   Indian Restaurant  0.02


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0    Airport Terminal  0.14
1     Airport Service  0.14
2     Harbor / Marina  0.07
3         Coffee Shop  0.07
4    Sculpture Garden  0.07
5            Boutique  0.07
6            

In [108]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Clustering 

In [129]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Bar,Restaurant,Hotel,Asian Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Steakhouse,Café,Cheese Shop,Restaurant
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Harbor / Marina,Boutique,Sculpture Garden,Boat or Ferry,Bar,Coffee Shop,Airport Lounge,Airport Gate
3,"Cabbagetown,St. James Town",Coffee Shop,Bakery,Italian Restaurant,Café,Pizza Place,Pub,Restaurant,Sandwich Place,Liquor Store,Jewelry Store
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Fried Chicken Joint,Burger Joint,Ice Cream Shop,Bubble Tea Shop,Salad Place,Gym / Fitness Center


In [130]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 6

dt_grouped_clustering = dt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 5, 4, 5, 0, 3, 2, 5, 0, 0])

In [131]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dt_merged = dt_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

In [126]:
dt_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Café,Mexican Restaurant,Theater,Breakfast Spot,Restaurant,Farmers Market
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Diner,Sporting Goods Shop,Ramen Restaurant,Bookstore,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Restaurant,Italian Restaurant,Hotel,Bakery,Beer Bar,Gastropub,Cocktail Bar,Clothing Store
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Steakhouse,Café,Cheese Shop,Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Fried Chicken Joint,Burger Joint,Ice Cream Shop,Bubble Tea Shop,Salad Place,Gym / Fitness Center
5,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568,0,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Bar,Restaurant,Hotel,Asian Restaurant,Burger Joint
6,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,0,Coffee Shop,Hotel,Aquarium,Café,Scenic Lookout,Fried Chicken Joint,Brewery,Plaza,Pizza Place,Baseball Stadium
7,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Restaurant,Bar,Deli / Bodega,Gastropub,Italian Restaurant,American Restaurant,Bakery
8,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Seafood Restaurant,Italian Restaurant,Gym,Steakhouse
9,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4,Grocery Store,Café,Park,Baby Store,Nightclub,Convenience Store,Restaurant,Diner,Italian Restaurant,Coffee Shop


In [132]:
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['Neighbourhood'], dt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [137]:
dt_merged.loc[dt_merged['Cluster Labels'] == 0, dt_merged.columns[[2] + list(range(6, dt_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Fried Chicken Joint,Burger Joint,Ice Cream Shop,Bubble Tea Shop,Salad Place,Gym / Fitness Center
5,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Bar,Restaurant,Hotel,Asian Restaurant,Burger Joint
6,"Harbourfront East,Toronto Islands,Union Station",Coffee Shop,Hotel,Aquarium,Café,Scenic Lookout,Fried Chicken Joint,Brewery,Plaza,Pizza Place,Baseball Stadium
7,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Bar,Deli / Bodega,Gastropub,Italian Restaurant,American Restaurant,Bakery
8,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Seafood Restaurant,Italian Restaurant,Gym,Steakhouse
17,"First Canadian Place,Underground city",Coffee Shop,Café,Hotel,Steakhouse,Restaurant,Seafood Restaurant,Bar,Gastropub,Gym,Deli / Bodega


In [138]:
dt_merged.loc[dt_merged['Cluster Labels'] == 1, dt_merged.columns[[2] + list(range(6, dt_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Rosedale,Park,Trail,Playground,Building,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [139]:
dt_merged.loc[dt_merged['Cluster Labels'] == 2, dt_merged.columns[[2] + list(range(6, dt_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Christie,Grocery Store,Café,Park,Baby Store,Nightclub,Convenience Store,Restaurant,Diner,Italian Restaurant,Coffee Shop


In [140]:
dt_merged.loc[dt_merged['Cluster Labels'] == 3, dt_merged.columns[[2] + list(range(6, dt_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"Harbord,University of Toronto",Café,Bakery,Bookstore,Sandwich Place,Japanese Restaurant,Italian Restaurant,Bar,Restaurant,Dessert Shop,Comfort Food Restaurant
14,"Chinatown,Grange Park,Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Bakery,Donut Shop


In [141]:
dt_merged.loc[dt_merged['Cluster Labels'] == 4, dt_merged.columns[[2] + list(range(6, dt_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Harbor / Marina,Boutique,Sculpture Garden,Boat or Ferry,Bar,Coffee Shop,Airport Lounge,Airport Gate


In [142]:
dt_merged.loc[dt_merged['Cluster Labels'] == 5, dt_merged.columns[[2] + list(range(6, dt_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbourfront,Regent Park",Coffee Shop,Pub,Park,Bakery,Café,Mexican Restaurant,Theater,Breakfast Spot,Restaurant,Farmers Market
1,"Ryerson,Garden District",Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Diner,Sporting Goods Shop,Ramen Restaurant,Bookstore,Pizza Place
2,St. James Town,Café,Coffee Shop,Restaurant,Italian Restaurant,Hotel,Bakery,Beer Bar,Gastropub,Cocktail Bar,Clothing Store
3,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Steakhouse,Café,Cheese Shop,Restaurant
11,"Cabbagetown,St. James Town",Coffee Shop,Bakery,Italian Restaurant,Café,Pizza Place,Pub,Restaurant,Sandwich Place,Liquor Store,Jewelry Store
12,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Café,Hotel,Mediterranean Restaurant,Gym,Men's Store
16,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Restaurant,Café,Beer Bar,Hotel,Seafood Restaurant,Cocktail Bar,Italian Restaurant,Fast Food Restaurant,Art Gallery
